In [ ]:
!pip install -q pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install highspy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00


In [ ]:
import requests
import pickle

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import folium
from folium import plugins

In [ ]:
import geopandas as gpd

In [ ]:
import pyomo.environ as pyo
from pyomo.contrib.appsi.solvers.highs import Highs

In [ ]:
# Get the url of the pkl file
warehouses = "https://github.com/paragsiddique/computational_logistics/raw/main/SCND_ch_02/warehouses.pkl"
customers = "https://github.com/paragsiddique/computational_logistics/raw/main/SCND_ch_02/customers.pkl"
customer_demands = "https://github.com/paragsiddique/computational_logistics/raw/main/SCND_ch_02/customer_demands.pkl"
distance = "https://github.com/paragsiddique/computational_logistics/raw/main/SCND_ch_02/distance.pkl"

data_list = [warehouses, customers, customer_demands, distance]
dict_name = ['warehouses', 'customers', 'customer_demands', 'distance']

als_athletic_data = {}
for i in range(len(data_list)):
    # Make a request to the url
    response = requests.get(data_list[i])
    # Decode the response body
    data = pickle.loads(response.content)
    als_athletic_data[dict_name[i]] = data

In [ ]:
df_customer = pd.DataFrame(list(als_athletic_data['customers'].values()), columns=['Facility Name', 'City', 'State', 'Zip', 'Latitude', 'Longitude'])
df_customer['Facility Name'] = 'CUST_'+df_customer['Facility Name']+'_'+df_customer['State']
df_customer['Demand']  = list(als_athletic_data['customer_demands'].values())
df_customer.head()

,Facility Name,City,State,Zip,Latitude,Longitude,Demand
0,CUST_Akron_OH,Akron,OH,,41.08,-81.52,205375
1,CUST_Albuquerque_NM,Albuquerque,NM,,35.12,-106.62,535923
2,CUST_Alexandria_VA,Alexandria,VA,,38.82,-77.09,147786
3,CUST_Amarillo_TX,Amarillo,TX,,35.20,-101.82,190449
4,CUST_Anaheim_CA,Anaheim,CA,,33.84,-117.87,342336


In [ ]:
demand_by_state = df_customer.groupby('State').agg(total_demand=('Demand', 'sum')).reset_index()
demand_by_state.head()

,State,total_demand
0,AL,802508
1,AR,192247
2,AZ,3840961
3,CA,15464159
4,CO,1746913


In [ ]:
df_warehouse = pd.DataFrame(list(als_athletic_data['warehouses'].values()), columns=['Facility Name', 'City', 'State', 'Zip', 'Latitude', 'Longitude'])
df_warehouse['Facility Name'] = 'WH_'+df_warehouse['Facility Name']+'_'+df_customer['State']
df_warehouse.head()

,Facility Name,City,State,Zip,Latitude,Longitude
0,WH_Allentown_OH,Allentown,PA,18101,40.602812,-75.470433
1,WH_Atlanta_NM,Atlanta,GA,30301,33.753693,-84.389544
2,WH_Baltimore_VA,Baltimore,MD,21201,39.294398,-76.622747
3,WH_Boston_TX,Boston,MA,02101,42.360970,-71.053440
4,WH_Chicago_CA,Chicago,IL,60602,41.883310,-87.624713


In [ ]:
current_wh_loc = (df_warehouse[-1:]['Latitude'].item(), df_warehouse[-1:]['Longitude'].item())
f = folium.Figure(width=1000, height=600)
#m = geemap.Map().add_to(f)
wh_cust_map = folium.Map(location=current_wh_loc, zoom_start=5).add_to(f)



my_icon = folium.Icon(color='red')

folium.Marker(
    location=current_wh_loc,
    tooltip=df_warehouse[-1:]['Facility Name'].item(),
    icon=my_icon,
).add_to(wh_cust_map)

for wh in range(len(df_warehouse)-1):
    current_wh_loc = (df_warehouse.loc[wh]['Latitude'].item(), df_warehouse.loc[wh]['Longitude'].item())
    folium.Marker(
    location=current_wh_loc,
    tooltip=df_warehouse.loc[wh]['Facility Name'],
    icon=folium.Icon(color='blue', icon='star'),
    ).add_to(wh_cust_map)


for cust in range(len(df_customer)):
    current_cust_loc = (df_customer.loc[cust]['Latitude'].item(), df_customer.loc[cust]['Longitude'].item())
    folium.CircleMarker(
    location=current_cust_loc,
    tooltip=df_customer.loc[cust]['Facility Name'],
    radius=5,
    color='green',
	fill_opacity=1.0,
	fill_color='green'
    ).add_to(wh_cust_map)


wh_cust_map

In [ ]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/main/examples/data"
)
state_geo = f"{url}/us-states.json"

In [ ]:
# We read the file and print it.
geoJSON_df = gpd.read_file(state_geo)
geoJSON_df.head()

,id,name,geometry
0,AL,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,AK,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,AZ,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
3,AR,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
4,CA,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."


In [ ]:
states_list = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
df_states = pd.DataFrame({'State':states_list})
df_demand = pd.merge(df_states, demand_by_state, on='State', how='left')
df_demand = df_demand.fillna(0)
df_demand.head()

,State,total_demand
0,AL,802508.0
1,AK,0.0
2,AZ,3840961.0
3,AR,192247.0
4,CA,15464159.0


In [ ]:
f = folium.Figure(width=1000, height=600)
m = folium.Map(location=[38.2527, -85.7585], zoom_start=4).add_to(f)
folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=df_demand,
    columns=["State", "total_demand"],
    key_on="feature.id",
    fill_color="YlOrRd",
    fill_opacity=0.95,
    line_opacity=0.25,
    legend_name="Demand",
).add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
df_distance_matrix = pd.DataFrame(als_athletic_data['distance'].items(), columns=['OD_num', 'Distance'])
df_distance_matrix

,OD_num,Distance
0,"(1, 1)",374
1,"(1, 2)",2041
2,"(1, 3)",177
3,"(1, 4)",1742
4,"(1, 5)",2777
...,...,...
5195,"(25, 196)",836
5196,"(25, 197)",464
5197,"(25, 198)",674
5198,"(25, 199)",1175


In [ ]:
wh_dict = {}
for i, x in enumerate(df_warehouse['Facility Name']):
    wh_dict[i+1] = x

cust_dict = {}
for i, x in enumerate(df_customer['Facility Name']):
    cust_dict[i+1] = x

In [ ]:
def dist_matrix_map_func(row):
    wh_indx = row['OD_num'][0]
    wh_name = wh_dict[wh_indx]
    cust_indx = row['OD_num'][1]
    cust_name = cust_dict[cust_indx]
    return (str(wh_name), str(cust_name))

df_distance_matrix['OD_text'] = df_distance_matrix.apply(lambda row: dist_matrix_map_func(row), axis=1)

In [ ]:
df_distance_matrix

,OD_num,Distance,OD_text
0,"(1, 1)",374,"(WH_Allentown_OH, CUST_Akron_OH)"
1,"(1, 2)",2041,"(WH_Allentown_OH, CUST_Albuquerque_NM)"
2,"(1, 3)",177,"(WH_Allentown_OH, CUST_Alexandria_VA)"
3,"(1, 4)",1742,"(WH_Allentown_OH, CUST_Amarillo_TX)"
4,"(1, 5)",2777,"(WH_Allentown_OH, CUST_Anaheim_CA)"
...,...,...,...
5195,"(25, 196)",836,"(WH_St. Louis_TX, CUST_Washington_DC)"
5196,"(25, 197)",464,"(WH_St. Louis_TX, CUST_Wichita_KS)"
5197,"(25, 198)",674,"(WH_St. Louis_TX, CUST_Winston-Salem_NC)"
5198,"(25, 199)",1175,"(WH_St. Louis_TX, CUST_Worcester_MA)"


In [ ]:
od_dist = dict(zip(df_distance_matrix['OD_text'], df_distance_matrix['Distance']))

**Parameters**
* $J$ Customers
* $I$ Facilities
* $P$ Facility limit
* $d_{j}$ Demand for customer $j$
* $dist_{i, j}$ Distance from facility $i$ to customer $j$

**Decision Variables**
* $x_{i} = 1$ if facility  $i$ is elected to be used, else $0$
* $y_{i, j} = 1$ if facility  $i$ serves customer $j$, else $0$



\begin{equation*}
\begin{array}{ll@{}ll}
\text{minimize}  & \displaystyle \sum\limits_{i\in I} \sum\limits_{j \in J} dist_{i, j}d_{j}y_{i, j} &\\
\text{subject to}& \displaystyle \sum\limits_{i\in I} y_{i, j} = 1  & \forall j \in J \\
& \displaystyle \sum\limits_{i\in I} x_{i} = P \\
& \displaystyle \sum\limits_{i\in I} y_{i,j} \leq  x_{i} & \forall i \in I, \forall j \in J \\
&x_{j} \in \{0,1\}, &\forall i \in I  \\
&y_{i,j} \in \{0,1\}, &\forall i \in I, \forall j \in J
\end{array}
\end{equation*}

In [ ]:
def p_facility_location_model(warehouses, customers, demand, od_dist, max_warehouse):

    model_name = f'als_athletics_{max_warehouse}'

    model=pyo.ConcreteModel(model_name)

    model.I = pyo.Set(initialize=warehouses)
    model.J = pyo.Set(initialize=customers)

    #Demand
    model.d = pyo.Param(model.J, initialize=dict(zip(customers, demand)))
    #Distance
    model.c = pyo.Param(model.I, model.J, initialize=od_dist)

    #if the warehouse i is selected to be opened
    model.x = pyo.Var(model.I, domain=pyo.Binary)
    #if the warehouse i serves customer j
    model.y = pyo.Var(model.I, model.J, domain=pyo.Binary)

    #every customer must be fully served
    def customer_fully_served(model, j):
        serve_customer_j_fully = sum(model.y[i, j] for i in model.I)
        return serve_customer_j_fully == 1
    model.constraint_customer_fully_served = pyo.Constraint(model.J, rule=customer_fully_served)

    # facility limit
    def warehouse_limit(model):
        warehouse_count = sum(model.x[i] for i in model.I)
        return warehouse_count == max_warehouse
    model.constraint_wh_limit = pyo.Constraint(rule=warehouse_limit)

    # if a warehouse serves a customer, then that warehouse must be considered “open”
    def wh_open_logic(model, i, j):
        return model.y[i, j] <= model.x[i]
    model.constraint_wh_open_logic = pyo.Constraint(model.I, model.J, rule=wh_open_logic)

    def obj_min_cost(model):
        total_cost = sum(model.d[j]*model.c[i, j]*model.y[i, j] for i in model.I for j in model.J)
        return total_cost
    model.obj_min_cost = pyo.Objective(sense=pyo.minimize, rule=obj_min_cost)

    #solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc')
    #solver = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')
    solver = Highs()
    solver.highs_options = {"time_limit": 500}
    result = solver.solve(model)
    #model.display()

    optimal_cost = model.obj_min_cost()

    avg_demand_wgtd_dist = optimal_cost/sum(demand)

    pre_optimal_locations= []
    for i in model.x:
        if model.x[i].value>0:
            pre_optimal_locations.append((str(model.x[i]), model.x[i].value))

    optimal_locations = [x[0][2:-1] for x in pre_optimal_locations]

    pre_optimal_arcs = []
    for i in model.y:
        if model.y[i].value>0:
            pre_optimal_arcs.append((str(model.y[i]), model.y[i].value))

    optimal_arcs = [tuple((x[0][2:-1]).split(",")) for x in pre_optimal_arcs]

    return model_name, optimal_cost, avg_demand_wgtd_dist, optimal_locations, optimal_arcs

In [ ]:
def map_plot_func(opt_result):

    #df_map_init_loc = df_warehouse[df_warehouse['Facility Name']==opt_result[3][0]]
    #initial_loc = (df_map_init_loc['Latitude'].item(), df_map_init_loc['Longitude'].item())
    initial_loc = (39.0997, -94.5786)

    f = folium.Figure(width=1000, height=600)
    map_plot = folium.Map(location=initial_loc, zoom_start=4).add_to(f)

    for wh in opt_result[3]:
        current_wh =  df_warehouse[df_warehouse['Facility Name']==wh]
        current_wh_loc = (current_wh['Latitude'].item(), current_wh['Longitude'].item())
        folium.Marker(
        location=current_wh_loc,
        tooltip=wh,
        icon=folium.Icon(color='red'),
        ).add_to(map_plot)

    for cust in customers:
        current_customer =  df_customer[df_customer['Facility Name']==cust]
        current_cust_loc = (current_customer['Latitude'].item(), current_customer['Longitude'].item())
        folium.CircleMarker(
        location=current_cust_loc,
        tooltip=cust,
        radius=5,
        color='green',
        fill_opacity=1.0,
        fill_color='green'
        ).add_to(map_plot)

    for arc in opt_result[4]:
        wh_lat_lon=(df_warehouse[df_warehouse['Facility Name']==arc[0]]['Latitude'].item(),
                    df_warehouse[df_warehouse['Facility Name']==arc[0]]['Longitude'].item())

        cust_lat_lon=(df_customer[df_customer['Facility Name']==arc[1]]['Latitude'].item(),
                      df_customer[df_customer['Facility Name']==arc[1]]['Longitude'].item())

        coordinates = [wh_lat_lon, cust_lat_lon]
        folium.PolyLine(coordinates, color="blue", weight=1.5, opacity=0.8).add_to(map_plot)

    return map_plot

In [ ]:
max_warehouse = 2
warehouses = list(df_warehouse['Facility Name'])
customers = list(df_customer['Facility Name'])
demand = list(df_customer['Demand'])
od_distance = od_dist
als_athletics_2 = p_facility_location_model(warehouses, customers, demand, od_distance, max_warehouse)
map_plot_func(als_athletics_2)

In [ ]:
max_warehouse = 5
warehouses = list(df_warehouse['Facility Name'])
customers = list(df_customer['Facility Name'])
demand = list(df_customer['Demand'])
od_distance = od_dist
als_athletics_5 = p_facility_location_model(warehouses, customers, demand, od_distance, max_warehouse)
map_plot_func(als_athletics_5)

In [ ]:
map_plot_func(als_athletics_2)